### Introduction

This lab environment allows access to the following models:

- A selection of Amazon Bedrock models.
- The Flan T5 Base model.

#### Installing, Upgrading, and Importing Dependencies

The following cells use the `pip` Python package manager to install `boto3` and `pandas` before importing them.

In [ ]:
!pip install --upgrade boto3 pandas

In [55]:
import boto3
import json
from IPython.display import JSON
region='us-west-2'

#### Bedrock

The following is an example of invoking the Nova Micro model using the Amazon Bedrock API. 

In [68]:
def invoke_bedrock_model(model_id, prompt):
    bedrock = boto3.client("bedrock-runtime", region_name=region)

    body = {
        "schemaVersion": "messages-v1",
        "messages": [{ 'role': "user", 'content': [{'text': prompt}]}],
    }

    response = bedrock.invoke_model(
        modelId=model_id,
        accept="application/json",
        contentType="application/json",
        body=json.dumps(body)
    )

    return response['body'].read()



In [ ]:
response = invoke_bedrock_model("us.amazon.nova-micro-v1:0", "Explain the main differences between supervised and unsupervised learning.")
JSON(json.loads(response), expanded=True)

*Notes*:

- You can use a different model by supplying a different `model_id`.
- Different models use different body formats. You will need to modify the `invoke_bedrock_model` to use a different model.
- Options such as `top_k` and `temperature` can be added to the `body`.

#### Bedrock with a Streaming Response

The following is an example of invoking the Nova Micro Amazon Bedrock model and processing a streaming response.

In [98]:
def invoke_bedrock_model_with_streaming(model_id, prompt):
    bedrock = boto3.client("bedrock-runtime", region_name=region)

    body = {
        "schemaVersion": "messages-v1",
        "messages": [{ 'role': "user", 'content': [{'text': prompt}]}],
    }

    response = bedrock.invoke_model_with_response_stream(
        modelId=model_id,
        accept="application/json",
        contentType="application/json",
        body=json.dumps(body)
    )

    for event in response.get('body', []):
        #print(event)
        chunk = event.get('chunk', [])
        #print(chunk)
        if chunk:
            chunk_json = json.loads(chunk.get('bytes').decode())
            #print(json.dumps(chunk_json, indent=2, ensure_ascii=False))
            print(chunk_json.get("contentBlockDelta", {}).get("delta", {}).get("text", ""), end="")


In [ ]:
invoke_bedrock_model_with_streaming("us.amazon.nova-micro-v1:0", "Explain the main differences between supervised and unsupervised learning.")

#### SageMaker Flan-T5-Base

The following is an example of invoking the [flan-t5-base](https://huggingface.co/google/flan-t5-base) model using the Amazon SageMaker AI API.

In [39]:
def invoke_sagemaker_endpoint(endpoint_name, request_data):
    sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)
    payload = json.dumps(request_data)

    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",  # The MIME type of your request payload
        Body=payload
    )

    return response["Body"].read().decode("utf-8")

In [ ]:
endpoint_name = "qa-flan-t5-base"
response = invoke_sagemaker_endpoint(endpoint_name=endpoint_name, request_data={"inputs": "Explain the main differences between supervised and unsupervised learning."})
JSON(json.loads(response), expanded=True)

*Notes*:

- The Flan T5 Base model has been deployed on a `ml.g5.2xlarge` instance.

### Accessing the Billsum Dataset

The following cells download a copy of the [FiscalNote/billsum](https://huggingface.co/datasets/FiscalNote/billsum) dataset, extracts it, and loads it into a Pandas data frame.

In [ ]:
!curl -sL "https://assets.labs.platform.qa.com/stepstone-ai-academy-third-hackathon/billsum.zip" -o ./billsum.zip
!unzip -n billsum.zip

In [ ]:
import pandas as pd
df = pd.read_parquet("data/train-00000-of-00001.parquet")
print(df.head())